In [31]:
import numpy as np
import pandas as pd
from typing import Union

In [32]:
def calc_bbox(array: np.ndarray):
    t = array[0,0]
    pos_array = array[:,1:]
    # 3d case
    if pos_array.shape[1] == 3:
        
        miny, minx, minz = np.min(pos_array, axis = 0)
        maxy, maxx, maxz = np.max(pos_array, axis = 0)
        return np.array([[t, miny, minx, minz], 
                        [t, miny, minx, maxz],
                        [t, miny, maxx, maxz],
                        [t, miny, maxx, minz],
                        [t, maxy, maxx, minz],
                        [t, maxy, minx, minz],
                        [t, maxy, minx, maxz],
                        [t, maxy, maxx, maxz],
                        ])
    # 2d case
    miny, minx = np.min(pos_array, axis = 0)
    maxy, maxx = np.max(pos_array, axis = 0)
    return np.array([[t ,miny, minx],
                    [t, miny, maxx],
                    [t, maxy, maxx], 
                    [t, maxy, minx]])

def get_bbox_2d(df: pd.DataFrame, frame: str, xcol: str, ycol: str, zcol: Union[str, None]):
    df = df.sort_values([frame])
    if zcol == 'None':
        array_tpos = df[[frame, ycol, xcol]].to_numpy()
    else: 
        array_tpos = df[[frame, ycol, xcol, zcol]].to_numpy()
    array_tpos = array_tpos[~np.isnan(array_tpos).any(axis=1)]
    # split array into list of arrays, one for each collid/timepoint combination
    grouped_array = np.split(
        array_tpos, np.unique(array_tpos[:, 0], axis=0, return_index=True)[1][1:]
    )
    # calc bbox for every array in the list
    bbox = [calc_bbox(i) for i in grouped_array]
    text_size = 10
    bbox_size = 5
    text_parameters = {
        'text': 'collid: {label}',
        'size': text_size,
        'color': 'white',
        'anchor': 'upper_left',
        'translation': [-3, 0]
    }
    bbox_layer = {}
    bbox_layer['properties'] = {}
    bbox_layer['properties']['label'] = colid
    bbox_layer['text'] = text_parameters
    bbox_layer['face_color'] = 'transparent'
    bbox_layer['edge_color'] = 'red'
    bbox_layer['edge_width'] = bbox_size
    bbox_layer['name'] = 'event_boundingbox'
    bbox_layer['shape_type'] = 'polygon'
    return bbox, bbox_layer

def get_bbox_3d(df: pd.DataFrame, frame: str, xcol: str, ycol: str, zcol: Union[str, None]):
    df = df.sort_values([frame])
    array_tpos = df[[frame, ycol, xcol, zcol]].to_numpy()
    array_tpos = array_tpos[~np.isnan(array_tpos).any(axis=1)]
    # split array into list of arrays, one for each collid/timepoint combination
    grouped_array = np.split(
        array_tpos, np.unique(array_tpos[:, 0], axis=0, return_index=True)[1][1:]
    )
    # calc bbox for every array in the list
    bbox = [calc_bbox(i) for i in grouped_array]
    dataFaces = []
    vertices_count = 0
    data_colors = []
    face = np.array([[3, 5, 4],
       [3, 5, 0],
       [3, 1, 2],
       [3, 1, 0],
       [7, 3, 2],
       [7, 3, 4],
       [6, 1, 0],
       [6, 5, 0],
       [6, 1, 2],
       [6, 7, 2],
       [6, 5, 4],
       [6, 7, 4]])
    for i, value in enumerate(bbox):
            dataFaces.append(np.add(face, vertices_count))
            vertices_count += len(bbox[i])
    out_faces = np.concatenate(dataFaces)
    # color_array = np.array(data_colors)
    bbox = np.concatenate(bbox)
    data_colors = np.array([1 for i in range(bbox.shape[0])])
    return (bbox, out_faces, data_colors)


In [38]:
df = pd.read_csv('C:/Users/benig/Desktop/arcos_data_3d.csv')
df_filtered = df[df['collid'] == 4]

In [49]:
bbox = get_bbox_3d(df_filtered, 'time', 'posx', 'posy', 'posz')

In [40]:
import napari
viewer = napari.Viewer()

C:\Users\benig\miniconda3\envs\arcos\lib\site-packages\napari\plugins\_plugin_manager.py:533: UserWarning: Plugin 'napari-features' has already registered a function widget 'features' which has now been overwritten
  warn(message=warn_message)


In [36]:
# if 'event_boundingbox' in viewer.layers:
#     viewer.layers.remove('event_boundingbox')
# viewer.add_shapes(bbox[0], hull.simplices)

In [53]:
print(bbox[1])

[[ 3  5  4]
 [ 3  5  0]
 [ 3  1  2]
 [ 3  1  0]
 [ 7  3  2]
 [ 7  3  4]
 [ 6  1  0]
 [ 6  5  0]
 [ 6  1  2]
 [ 6  7  2]
 [ 6  5  4]
 [ 6  7  4]
 [11 13 12]
 [11 13  8]
 [11  9 10]
 [11  9  8]
 [15 11 10]
 [15 11 12]
 [14  9  8]
 [14 13  8]
 [14  9 10]
 [14 15 10]
 [14 13 12]
 [14 15 12]
 [19 21 20]
 [19 21 16]
 [19 17 18]
 [19 17 16]
 [23 19 18]
 [23 19 20]
 [22 17 16]
 [22 21 16]
 [22 17 18]
 [22 23 18]
 [22 21 20]
 [22 23 20]
 [27 29 28]
 [27 29 24]
 [27 25 26]
 [27 25 24]
 [31 27 26]
 [31 27 28]
 [30 25 24]
 [30 29 24]
 [30 25 26]
 [30 31 26]
 [30 29 28]
 [30 31 28]
 [35 37 36]
 [35 37 32]
 [35 33 34]
 [35 33 32]
 [39 35 34]
 [39 35 36]
 [38 33 32]
 [38 37 32]
 [38 33 34]
 [38 39 34]
 [38 37 36]
 [38 39 36]
 [43 45 44]
 [43 45 40]
 [43 41 42]
 [43 41 40]
 [47 43 42]
 [47 43 44]
 [46 41 40]
 [46 45 40]
 [46 41 42]
 [46 47 42]
 [46 45 44]
 [46 47 44]
 [51 53 52]
 [51 53 48]
 [51 49 50]
 [51 49 48]
 [55 51 50]
 [55 51 52]
 [54 49 48]
 [54 53 48]
 [54 49 50]
 [54 55 50]
 [54 53 52]
 [54

In [37]:
if 'event_boundingbox' in viewer.layers:
    viewer.layers.remove('event_boundingbox')
viewer.add_surface(bbox, colormap='red')

<Surface layer 'bbox' at 0x25daa488910>

In [20]:
hull.simplices

array([[3, 5, 4],
       [3, 5, 0],
       [3, 1, 2],
       [3, 1, 0],
       [7, 3, 2],
       [7, 3, 4],
       [6, 1, 0],
       [6, 5, 0],
       [6, 1, 2],
       [6, 7, 2],
       [6, 5, 4],
       [6, 7, 4]], dtype=int32)